In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import os
import sys

# --- INPUTS DINÁMICOS DESDE ARGUMENTO ---
# El usuario pasa como argumento la ruta base donde están/van los archivos
default_path = r"/mnt/c/Users/Administrador/Downloads"

try:
    passed_path = sys.argv[1]
    if os.path.isdir(passed_path):
        base_path = passed_path
    else:
        print(f"La ruta proporcionada no es válida: {passed_path}. Usando ruta por defecto.")
        base_path = default_path
except IndexError:
    print("No se proporcionó ruta, usando la ruta por defecto.")
    base_path = default_path

# Asegurarse de que la carpeta existe
os.makedirs(base_path, exist_ok=True)

# Archivos fijos dentro de la ruta proporcionada
csv_path = os.path.join(base_path, "DatabankExport.csv")
config_path = os.path.join(base_path, "config.xml")
output_path = os.path.join(base_path, "config.cfx")

# --- PARÁMETROS Y FUNCIONES ---
def process_indicators(column_name):
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"El archivo '{csv_path}' no existe. Verifica la ruta base o el nombre del archivo.")
    df = pd.read_csv(csv_path, delimiter=';', encoding='utf-8')
    expanded = df.assign(**{column_name: df[column_name].str.split(',')}).explode(column_name)
    grouped = expanded.groupby(column_name).agg({
        'Edge Ratio': 'mean',
        'Ret/DD Ratio': 'mean',
        'Profit factor': 'mean',
        column_name: 'count'
    }).rename(columns={
        'Edge Ratio': 'Avg_EdgeRatio',
        'Ret/DD Ratio': 'Avg_RetDD',
        'Profit factor': 'Avg_ProfitFactor',
        column_name: 'Count'
    })
    grouped['Score'] = (
        grouped['Avg_EdgeRatio'] * 20 +
        grouped['Avg_RetDD'] * 15 +
        grouped['Avg_ProfitFactor'] * 15 +
        grouped['Count'] * 50
    )
    return grouped.sort_values(by='Score', ascending=False).head(15).index.tolist()

# Obtener los nombres de indicadores más prometedores
top_entry_indicators = process_indicators('Entry indicators')
top_price_indicators = process_indicators('Price indicators')
pf = pd.DataFrame({
    'Top Entry Indicators': top_entry_indicators,
    'Top Price Indicators': top_price_indicators
})
display(pf)
essential_indicator_operators = [
    "CrossesAbove", "CrossesBelow", "IndicatorAboveMA", "IndicatorBelowMA",
    "IndicatorCrossesAboveMA", "IndicatorCrossesBelowMA",
    "IsRising", "IsFalling", "IsGreater", "IsGreaterOrEqual",
    "IsLower", "IsLowerOrEqual", "Equals", "NotEquals",
    "IsGreaterPercentil", "IsLowerPercentil", "IsGreaterCount", "IsLowerCount", "Not"
]

def matches_suffix(key, indicators_list):
    return any(key.endswith(f".{ind}") for ind in indicators_list)

# --- XML MODIFICACIÓN ---
tree = ET.parse(config_path)
root = tree.getroot()
blocks_root = root.find(".//BuildingBlocks")

if blocks_root is not None:
    for block in blocks_root.findall("Block"):
        key = block.attrib.get("key", "")
        category = block.attrib.get("category", "").lower()

        # Entry indicators y operadores
        if category in ["indicators", "signals"]:
            if matches_suffix(key, top_entry_indicators) or key in essential_indicator_operators:
                block.set("use", "true")
            else:
                block.set("use", "false")

        # Price indicators
        elif category == "stoplimitblocks":
            if matches_suffix(key, top_price_indicators):
                block.set("use", "true")
            else:
                block.set("use", "false")

# Guardar XML temporal en la ruta base
temp_xml = os.path.join(base_path, "config_temp.xml")
tree.write(temp_xml, encoding="utf-8", xml_declaration=True)

# Comprimir en .cfx dentro de la ruta base
with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(temp_xml, arcname="config.xml")

# Eliminar el archivo temporal
os.remove(temp_xml)

La ruta proporcionada no es válida: --f=/run/user/0/jupyter/runtime/kernel-v354780febb24dc62db4f2b0e55a5a63bd74e20b1b.json. Usando ruta por defecto.


,Top Entry Indicators,Top Price Indicators
0,HighestInRange,BiggestRange
1,LowestInRange,BBRange
2,HeikenAshiLow,SmallestRange
3,HeikenAshiHigh,TrueRange
4,Lowest,BBWidthRatio
5,Highest,MTATR
6,KAMA,BarRange
7,High,ATR
8,Low,HighestInRange
9,SessionClose,Highest
